# <center> UE22CS342AA2 - Data Analytics </center>

# <center> Worksheet 4a: Collaborative Filtering </center>

<center> Designed by Amritha GK, PESU-25 </center>

<br>

- Student name: Mohul Y P
- SRN: PES1UG22CS360
- Section: F

Assume there exists a parallel universe and you are a Data Analyst intern at Parallel Earth Amazon. The company wants to test product recommendations and improve their product catalogue. For this exercise, you are tasked with the analysis of User Ratings for Health and Personal Care Products.



# Details of the dataset:

### Columns in this dataset:

- 'sl': serial number of records

- 'user_id': Unique identifier for users

- 'title': title of the review

- 'text': body of the review

- 'asin': Amazon Standard Identification Number (ASIN) is a ten-digit alphanumeric code that identifies products on Amazon. It's unique for each product and is assigned when you create a new product in Amazon's catalog.

- 'parent_asin': The abstraction of the variations is called the parent ASIN. It can't be purchased and hence is not associated with an offer.

- 'rating': Product rating (out of 5)

- 'timestamp': In milliseconds

- 'helpful_vote': Interactions of other users with this review

- 'verified_purchase': (True or False) Amazon has verified that the person writing the review purchased the product from Amazon, and didn't receive the product at a discount.



## Some conceptual information:

### Utility Matrix

This is an m x n matrix (m users, n items) that consists the rating / preference of each user for each item.

It is usually quite sparse as the rating values are usually present for very few user-item pairs.



### Neighborhood-based methods

The main idea with neighborhood-based methods is to leverage either user-user similarity or item-item similarity to make recommendations. These methods assume that similar users tend to have similar behaviors when rating items.

There are mainly two types:



1. **User-based collaborative filtering**: based on similarity between users.

2. **Item-based collaborative filtering**: based on similarity between items.



### Matrix factorization/decomposition

The utility matrix can be represented as a product of two smaller matrices, say U and V, representing users and items respectively.

## Install the required packages

In [3]:
# install.packages("recommenderlab")

# install.packages("tidyr")

## Load the required packages

In [4]:
# library(dplyr)

# library(tidyr)   

# library(recommenderlab) 

# library(ggplot2)

## Load and create dataframe

In [1]:
# Your code here

import pandas as pd

path = '/kaggle/input/health-and-personal-care-dataset/Worksheet-4a_Health_And_Personal_Care_Dataset.csv'

df = pd.read_csv(path)

# Print the column names
print(df.columns.tolist())

['Unnamed: 0', 'user_id', 'title', 'text', 'asin', 'parent_asin', 'rating', 'timestamp', 'helpful_vote', 'verified_purchase']


In [2]:
df.columns

Index(['Unnamed: 0', 'user_id', 'title', 'text', 'asin', 'parent_asin',
       'rating', 'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

## Perform EDA and Preprocess the dataset to the required format

### 1 point

State some observations that you made after performing EDA.

Feel free to remove any columns you think are not necessary for the following problems.

Some hints to help you get started:
- Check the summary, number of rows and columns
- Remove null values and duplicates
- Obtain a subset of the dataset with the required columns only.
- Create the utility matrix


In [3]:
# Your code here

# Drop the 'Unnamed: 0' column
df = df.drop(columns=['Unnamed: 0'])


In [4]:
# Print the column names 
print(df.columns.tolist())

['user_id', 'title', 'text', 'asin', 'parent_asin', 'rating', 'timestamp', 'helpful_vote', 'verified_purchase']


In [5]:
# Display basic summary statistics
print("Summary Statistics:")
print(df.describe())

Summary Statistics:
            rating     timestamp  helpful_vote
count  4276.000000  4.276000e+03   4276.000000
mean      4.284846  1.610122e+12      0.538354
std       0.986821  3.583338e+10      2.841823
min       1.000000  1.345324e+12      0.000000
25%       4.000000  1.592960e+12      0.000000
50%       5.000000  1.613443e+12      0.000000
75%       5.000000  1.633993e+12      0.000000
max       5.000000  1.683520e+12     93.000000


In [6]:
# Display basic data info
print("\nDataset Information:")
print(df.info())


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4276 entries, 0 to 4275
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            4276 non-null   object 
 1   title              4276 non-null   object 
 2   text               4276 non-null   object 
 3   asin               4276 non-null   object 
 4   parent_asin        4276 non-null   object 
 5   rating             4276 non-null   float64
 6   timestamp          4276 non-null   int64  
 7   helpful_vote       4276 non-null   int64  
 8   verified_purchase  4276 non-null   bool   
dtypes: bool(1), float64(1), int64(2), object(5)
memory usage: 271.6+ KB
None


In [7]:
# Checking for null values
print("\nNumber of Null Values in Each Column:")
print(df.isnull().sum())

# Dropping duplicates if any
df = df.drop_duplicates()

# Removing rows that have null values
df = df.dropna()

# Printing new number of rows and columns
print("\nNumber of Rows and Columns After Cleaning:")
print(df.shape)


Number of Null Values in Each Column:
user_id              0
title                0
text                 0
asin                 0
parent_asin          0
rating               0
timestamp            0
helpful_vote         0
verified_purchase    0
dtype: int64

Number of Rows and Columns After Cleaning:
(4276, 9)


In [8]:
# Display the first few rows of the cleaned data
print("\nCleaned Dataset Preview:")
print(df.head())


Cleaned Dataset Preview:
                        user_id                               title  \
0  AE23ZBUF2YVBQPH2NN6F5XSA3QYQ  The Soap Petals Are A Fun Addition   
1  AE23ZBUF2YVBQPH2NN6F5XSA3QYQ       Add More Water Than Suggested   
2  AE23ZBUF2YVBQPH2NN6F5XSA3QYQ               Not very comfortable.   
3  AE23ZBUF2YVBQPH2NN6F5XSA3QYQ                Smells Intoxicating!   
4  AE23ZBUF2YVBQPH2NN6F5XSA3QYQ                  More Perfume Scent   

                                                text        asin parent_asin  \
0  I'm using to purchasing bath bombs for nearly ...  B08D93ZSWY  B08D93ZSWY   
1  We live in the south. You need all the help wh...  B09GL3RDTD  B09GL3RDTD   
2  This is difficult to give a good review becaus...  B08RD257KT  B08RD257KT   
3  Oh man these smell so good! Lemongrass is my f...  B098WFX6TY  B098WFX6TY   
4  $12 for 6 bombs is pretty average when it come...  B08Z474Q6J  B08Z474Q6J   

   rating      timestamp  helpful_vote  verified_purchase  
0     

In [9]:
# Create the utility matrix: rows = user_id, columns = asin, values = rating
utility_matrix = df.pivot_table(index='user_id', columns='asin', values='rating')

# Check the shape and preview the utility matrix
print("Utility Matrix Shape:", utility_matrix.shape)
print("Utility Matrix Sample:\n", utility_matrix.head())

Utility Matrix Shape: (500, 880)
Utility Matrix Sample:
 asin                          B00009QJW6  B000F95A4W  B000G2BESO  B000OUY4T4  \
user_id                                                                        
AE23ZBUF2YVBQPH2NN6F5XSA3QYQ         NaN         NaN         NaN         NaN   
AE2O2C43KTYO4LXXGZWJZLE67GBQ         NaN         NaN         NaN         NaN   
AE2YCIHHZH57ABZB7EWDTCF3WPKA         NaN         NaN         NaN         NaN   
AE3335XF4PMHSXKTW5B7N7EALG3Q         NaN         NaN         NaN         NaN   
AE3IGJOPJP6LFXEJTIXFJVSJLILA         NaN         NaN         NaN         NaN   

asin                          B000WFG9JO  B002DXU1FQ  B004AWD1CS  B004C7MTLA  \
user_id                                                                        
AE23ZBUF2YVBQPH2NN6F5XSA3QYQ         NaN         NaN         NaN         NaN   
AE2O2C43KTYO4LXXGZWJZLE67GBQ         NaN         NaN         NaN         NaN   
AE2YCIHHZH57ABZB7EWDTCF3WPKA         NaN         NaN         N

In [10]:
df.columns

Index(['user_id', 'title', 'text', 'asin', 'parent_asin', 'rating',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

**Observations:**

The rating matrix is largely sparse. It has many missing values (NaN values). A majority of the products have been rated by only a small portion of users. Techniques such as collaborative filtering or matrix factorization are well-suited for predicting and filling in the missing ratings.
Imputation methods are necessary to effectively handle the absence of ratings (NaN values).

## Problem 1

### User-based collaborative filtering: 3 points (2+1)

1. As part of the test run of the new business strategy, you are supposed to pick a user (consider the user with user_id:"AE2O2C43KTYO4LXXGZWJZLE67GBQ") and recommend 5 products based on other similar users.

Remember to exclude the product(s) purchased by this user.

2. Which similarity metric did you use? Why?

In [11]:
# Your code here
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'df' is the DataFrame containing user-product ratings
# Target user
target_user = "AE2O2C43KTYO4LXXGZWJZLE67GBQ"

# Create a utility matrix where rows are users and columns are products (asin)
utility_matrix = df.pivot(index='user_id', columns='asin', values='rating')

# Replace NaN values with 0, since we are treating unrated products as 0
utility_matrix = utility_matrix.fillna(0)

# Compute cosine similarity between all users
cosine_sim = cosine_similarity(utility_matrix)

# Convert cosine_sim to a DataFrame for easier manipulation
cosine_sim_df = pd.DataFrame(cosine_sim, index=utility_matrix.index, columns=utility_matrix.index)

# Get the similar users for the target user (exclude the user themselves)
similar_users = cosine_sim_df[target_user].drop(target_user)

# Get the most similar users (top 5)
top_similar_users = similar_users.nlargest(5)

# Get the products rated by these similar users, excluding the products already rated by the target user
recommended_products = set()

for user in top_similar_users.index:
    user_ratings = utility_matrix.loc[user]
    rated_products = user_ratings[user_ratings > 0].index  # Products rated by the user
    recommended_products.update(rated_products)

# Exclude products already rated by the target user
target_user_rated = utility_matrix.loc[target_user][utility_matrix.loc[target_user] > 0].index
recommended_products.difference_update(target_user_rated)

# Display top 5 recommended products
recommended_products = list(recommended_products)[:5]
recommended_products


['B086862KGK', 'B08CKKLXHD', 'B07XSKJ6LC', 'B07Y361K2L', 'B09C4SPQVR']

**QUESTION 2:**

Cosine similarity was selected because it measures the similarity between two users' preferences by analyzing whether they liked the same products. It focuses on the pattern of ratings rather than their specific values.

## Problem 2

### 1 point

Consider the same user as in the previous problem.

Let's assume this user decides to purchase the first product among the recommendations above. What rating is the user likely to give to this product?

In [ ]:
# Your code here

# Get the index of the target user
target_user_id = "AE2O2C43KTYO4LXXGZWJZLE67GBQ"
user_index = utility_matrix.index.get_loc(target_user_id)

# Get the similarity matrix and ratings of similar users for the target product
similarity_scores = cosine_sim[user_index, :]  # Similarity scores of all users with the target user

# Extract the ratings for the target product from all users
product_ratings = utility_matrix['B07Y361K2L'].values

# Exclude the current user's own rating (or NaN if they haven't rated the product)
target_user_rating = product_ratings[user_index]

# Calculate weighted average of ratings by similar users (ignoring the current user’s own rating)
weighted_ratings = similarity_scores * product_ratings
predicted_rating = weighted_ratings.sum() / similarity_scores.sum()

predicted_rating

## Problem 3

### Item-based Collaborative Filtering: 2 points

Use the item-based method from the package "recommenderlab" to recommend top-5 products to the user "AE5IMGWRBJA7JQFBQTBK25HDYGVA".

In [13]:
!pip install scikit-surprise

In [14]:
# Your code here

from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

In [15]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split


# Define the reader for Surprise
reader = Reader(rating_scale=(1, 5))  # Assuming ratings are from 1 to 5

# Loading data into Surprise's Dataset
dataset = Dataset.load_from_df(df[['user_id', 'asin', 'rating']], reader)

# Splitting data into training and test sets
trainset, testset = train_test_split(dataset, test_size=0.2)

# Train the SVD model
svd = SVD()
svd.fit(trainset)

# Get top-5 product recommendations for a user
user_id = 'AE5IMGWRBJA7JQFBQTBK25HDYGVA'
all_products = df['asin'].unique()
rated_products = df[df['user_id'] == user_id]['asin']
unrated_products = [product for product in all_products if product not in rated_products]

# Predicting ratings for unrated products
predictions = [svd.predict(user_id, product).est for product in unrated_products]

# Getting top 5 recommended products (highest predicted ratings)
top_5_recommended = sorted(zip(unrated_products, predictions), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 recommended products:", [product for product, _ in top_5_recommended])

Top 5 recommended products: ['B085LHTW4G', 'B08VSBR4L8', 'B098JWCZHB', 'B08KGTL2QG', 'B09VCSTX43']


## Problem 4

### Long-tail problem: 2 points

1. Plot a graph of the products and their frequencies(in descending order of frequency). State your observations about the graph. 

2. Suggest 2 ways to overcome the long-tail problem of recommender systems.



List the links to the resources you referred to for your answer.

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
# Your code here

# product frequencies calculation
product_frequencies = df['asin'].value_counts()

# Plotting frequency distribution
plt.figure(figsize=(12, 6))
sns.barplot(x=product_frequencies.index[:50], y=product_frequencies.values[:50])  # top 50 products
plt.xticks(rotation=90)
plt.xlabel('Product ID')
plt.ylabel('Frequency')
plt.title('Top 50 Products by Frequency')
plt.show()

**QUESTION 2:**

Two approaches to address the long-tail problem:

Item-Item Similarity: Suggest items that are similar to those a user likes, regardless of their popularity.

Hybrid Systems: Combine user preferences with item attributes to recommend products, even if they haven't received many ratings.

## Problem 5

### 1 point

Do you see any shortcomings of Collaborative Filtering due to the dataset or other factors? What modifications would you introduce to achieve better results?

*Your answer here*

**ANSWER:**

Collaborative filtering struggles with sparse data, making it hard to find reliable similarities, leading to the cold start problem and a bias toward popular items. A hybrid approach combining collaborative and content-based filtering, along with implicit feedback like clicks or views, can improve recommendations and address these challenges.

**Challenges with Collaborative Filtering:**

*Sparsity:* Insufficient ratings for many products make accurate recommendations difficult.

*Cold Start:* New users or products with few ratings receive poor recommendations.

*Popularity Bias:* Tends to favor popular items, overlooking niche or less popular ones.

**Modifications to Improve Collaborative Filtering:**

*Matrix Factorization:* Identifies hidden patterns to improve recommendations, even with limited ratings.

*Hybrid Systems:* Combines user behavior with product attributes to enhance recommendations, especially for new users or items.

*Improved Similarity Measures:* Utilizes advanced methods to calculate similarities, making recommendations more effective for sparse data.

Kudos on completing the task! Hope you could learn a few new things along the way.

Here's an interesting approach that uses LLMs and the FAISS library to create a movie recommender system in python: https://www.youtube.com/watch?v=epidA1fBFtI&t=122s

Do explore the above resource as it might have a bunch of new techniques in store for you.